# Distributed Systems

## Lecture 1 Introduction
Distributed Systems: a group of cooperating computers communicating with each other over network to get some tasks done.

### Use cases
- storage for big website
- big data computation （e.g. MapReduce）

### Why distributed systems?
- parallelism
- fault tolerance
- physical reasons 
- security

### Challenges
- concurrency
- partial failure
- performance

### Infrasctructure
The goal is to build distributed applications that act as single systems and abstract away the distributedness.

#### Types
- storage
- communication
- computation

### Topics
- implementation 
  - RPC, threads, concurrency
- performance
  - scalability
    - full scalability: 2 times the computers / resources gets 2 times the throughput
    - rare to get full scalability due to system bottlenecks
  - fault tolerance
    - large-scale applications contain many components prone to failure -> 
    - the ability to perceive and mask a failure is required
    - what it means to be fault tolerant?
      - availability: the system can keep operating despite the failure of some of its component and provide undamaged service
      - recoverability: the system can continue operating after failures without loss of correctness
        - tools: 
          - non-volatile storage 
            - to store checkpoints or logs of the state of the system
            - challenges: management of NV systems due to expensiveness to update -> 
            - need clever ways to avoid writes to NV storage
          - replication
            - challenges: syncing the replicas
            - replicas should have different independent failure probability
  - consistency
    - operations
      - `put(k,v)`
      - `get(k) -> v`
    - challenges: 
      - more than one copy of the data in distributed systems ->
      - may end up with two versions of data if put operations fail for some copies of data
    - types of consistency
      - strong consistency: `get` operations return the most recent put
      - weak consistency: `get` operations do not guarante the most recent put and may return old values
        - why useful: avoid excessive communication costs 
          - why expensive: replicas are often far apart for independent failure probability

### Case study: MapReduce
  - context: Google wants create a framework that abstract away the low-level implementation details of distributed systems and allow non-specialists to write and run distributed computations
  - abstract view of MapReduce
    - input is split into a different files with `map` and `reduce` operations
  ```
    * Map: split the input into words and for every word it sees emits the word as the key and 1 as value, then group together all instances of the same keys
      
    Map(k, v)
      split v into words
        for each word w
          emit(w, "1")

    * Reduce: sum up the total number of instances for every key

    Reduce(k, v)
      emit(len(v))

    * Example of a MapReduce job:
      input 1 ('ab') -> `Map` -> [(a, 1), (b, 1)] 
      input 2 ('b') -> `Map` -> [(b, 1)]
      input 3 ('ac') -> `Map` -> [(a, 1), (c, 1)]
      
      -> 

      [(a, 1), (a, 1)] -> `Reduce` -> (a, 2)
      [(b, 1)] -> `Reduce` -> (b, 1)
      [(c, 1)] -> `Reduce` -> (c, 1)
  ```
- Q: Where does the data emitted by `Map` and `Reduce` functions go?
  - underneath the MapReduce system there is some big collection of servers (worker servers) and a single master server orchestrating the systems
    - the master server farms out `Map` invitations for a set of input files to worker servers
    - worker servers read the input files, then call the `Map` functions with the assigned input files
    - then worker will implement `emit` to write data to files to local disks to perform `Reduce` functions
    - `Reduce` emits write the output to a file in Google's cluster file service 
  - GFS (Google File Service): a cluster file system that runs on the exactly same set of worker servers for MapReduce
    - automatically splits up big files evenly across worker servers in 64MB chunks
    - can then launch map workers in parallel ->
    - result: increased read throughput
- Q: How does inputs get correctly passed to `Map` functions?
  - MapReduce worker process fetch the input stored in a GFS server
  - bottleneck: network throughput for the fetch operations
    - MapReduce and GFS servers communicate through some root Ethernet switch with limited throughput
    - each machine shares 50 mbit/sec -> small compared to disk and CPU
  - solutions: run GFS and MapReduce on the same machines
    - fetching: master servers finds the server that holds the input file and send the map function to the input file on the same machine for local reads
    - storing: stores the output of the map on the same disk
  - limitations:
    - shuffle: grouping of values under the same key still requires network operations -> expansive part of the MapReduce
    - sorting: outputs have the same size as the inputs -> large amount of data
  - modern data centers introduce multiple network switches -> no longer need to run MapReduce on the same machines
- Q: How to run another map-reduce operation on the output of a map-reduce?
  - outputs of map-reduce are stored on the GFS servers ->
  - another round of network operations needed for getting outputs of reduce to GFS